In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime

#C:\Users\mmach\Documents\dh\Ecobici
#Carga de df

df = pd.read_csv("DATA/recorridos-mayo-2019.csv", delimiter = ',')

In [3]:

#droppin Na
df.dropna(subset = ['long_estacion_origen','long_estacion_destino','lat_estacion_destino','lat_estacion_destino' , 'domicilio_estacion_origen', 'domicilio_estacion_destino' ], inplace = True)
df.drop(['genero_usuario'], axis = 1, inplace =  True) # has not values
df.rename(columns = {'Unnamed: 0' : 'ID'}, inplace =True)

In [4]:
#converting from string to datetime
df['origen_datetime'], df['destino_datetime'] = pd.to_datetime(df[ 'fecha_origen_recorrido']) ,  pd.to_datetime(df['fecha_destino_recorrido'])

#getting duration of each trip
df['duracion'] =df.destino_datetime - df.origen_datetime
df['duracion'] = round(df.duracion.dt.total_seconds() / 60,0).astype(int)

# getting the hour
df['hora'] = df.origen_datetime.dt.hour

In [6]:


# Day of week. Mon 0 Sun 6
df['dia_semana']= df.origen_datetime.dt.dayofweek
# month
df['mes'] = df.origen_datetime.dt.month
# workday or not
df['tipo_dia'] = np.where(df['dia_semana']<5, 'Laboral', 'Fin de Semana')

In [7]:
#categorizing times of the day

bins_momentos = [5,10,16,20,24]
momentos_labels = ["6 a 10hs","11 a 16hs", "17 a 20hs", "21 a 5hs" ]
df['momento_dia'] = pd.cut(df.hora,bins_momentos,labels = momentos_labels, ordered = False)
df['momento_dia'].fillna("21 a 5hs", inplace = True)

#categorizing trip duration

bins_duracion = [-1,5,15,30,45,df.duracion.max()+1]
duracion_labels = ['Hasta 5 minutos', 'Hasta 15 minutos', 'Hasta 30 minutos', 'Hasta 45 minutos', 'Desde 45 minutos']
df['duracion_viaje'] = pd.cut(df.duracion,bins_duracion,labels = duracion_labels, ordered = False)

In [8]:
#consolidating stations between origin and destiny stations available in the dataset
estaciones_origen = df[['id_estacion_origen','nombre_estacion_origen', 'long_estacion_origen','lat_estacion_origen', 'domicilio_estacion_origen']].drop_duplicates(subset = ['id_estacion_origen'])
estaciones_origen.rename(columns = {'id_estacion_origen' : 'id_estacion',
                            'nombre_estacion_origen' : 'nombre_estacion', 
                            'long_estacion_origen': 'long_estacion',
                            'lat_estacion_origen': 'lat_estacion',
                            'domicilio_estacion_origen' : 'domicilio_estacion'}, inplace =True)
estaciones_destino = df[['id_estacion_destino','nombre_estacion_destino', 'long_estacion_destino','lat_estacion_destino', 'domicilio_estacion_destino']].drop_duplicates(subset = ['id_estacion_destino'])
estaciones_destino.rename(columns = {'id_estacion_destino' : 'id_estacion',
                            'nombre_estacion_destino' : 'nombre_estacion', 
                            'long_estacion_destino': 'long_estacion',
                            'lat_estacion_destino': 'lat_estacion',
                            'domicilio_estacion_destino' : 'domicilio_estacion'}, inplace =True)
            
                          
df_estaciones = estaciones_origen.append(estaciones_destino)
df_estaciones = df_estaciones.drop_duplicates(subset = ['id_estacion']).copy()

In [9]:

#converting latlong columns into GeoDf
crs = {'init' : 'epsg:4326'}
comunas = gpd.read_file('DATA/CABA_comunas.geojson')

In [10]:
#extracting Comunas info
geo_point = gpd.GeoDataFrame(df_estaciones[['long_estacion', 'lat_estacion']], geometry=gpd.points_from_xy(df_estaciones.long_estacion, df_estaciones.lat_estacion), crs=crs)
comunas_joint = gpd.sjoin(geo_point, comunas, how = "left")
df_estaciones['comuna'] = comunas_joint['COMUNAS'].copy()

In [11]:
df_viajes = df.drop(['periodo', 'nombre_estacion_origen', 'domicilio_estacion_origen', 'fecha_origen_recorrido',
                    'fecha_destino_recorrido',  'nombre_estacion_destino', 'domicilio_estacion_destino' ], axis = 1)

df_viajes = df_viajes.merge(df_estaciones[['id_estacion', 'comuna']], left_on='id_estacion_origen', right_on='id_estacion', suffixes=('','_origen')).drop(columns = 'id_estacion')
df_viajes = df_viajes.merge(df_estaciones[['id_estacion', 'comuna']], left_on='id_estacion_destino', right_on='id_estacion', suffixes=('','_destino')).drop(columns = 'id_estacion')



In [12]:
#assigning of type of trip with respect to Comuna 1
def asignar_tipo_de_viaje(df,origen,destino):
    tipo = []
    for j in df.index:
        if origen[j] == 1:
            if destino[j] == 1:
                tipo.append("Internos Comuna 1") 
            else:
                tipo.append("Desde la Comuna 1") 
        elif destino[j] == 1:
            tipo.append("Hacia la Comuna 1") 
        else:
            tipo.append("Fuera de la Comuna 1") 
    return tipo 

In [13]:
df_viajes['tipo_de_viaje'] = asignar_tipo_de_viaje(df_viajes,df_viajes['comuna'],df_viajes['comuna_destino'])

In [14]:

#calculate bearing
#code from https://stackoverflow.com/questions/47659249/calculate-cardinal-direction-from-gps-coordinates-in-python
import math

def calcBearing (lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    bearing = math.atan2(x,y)   # use atan2 to determine the quadrant
    bearing = math.degrees(bearing)

    return bearing

def calcNSEW(lat1, long1, lat2, long2):
    #points = ["north", "north east", "east", "south east", "south", "south west", "west", "north west"]
    points = ["N", "NE", "E", "SE", "S", "SO", "O", "NO"]
    bearing = calcBearing(lat1, long1, lat2, long2)
    bearing += 22.5
    bearing = bearing % 360
    bearing = int(bearing / 45) # values 0 to 7
    NSEW = points [bearing]

    return NSEW
    
def calcGrados(lat1, long1, lat2, long2):
    #points = ["north", "north east", "east", "south east", "south", "south west", "west", "north west"]
    points = ["N", "NE", "E", "SE", "S", "SO", "O", "NO"]
    bearing = calcBearing(lat1, long1, lat2, long2)
    bearing += 22.5
    bearing = bearing % 360
    return bearing


In [15]:
#calculate bearing 

#getting cardinal bearing
df_viajes['orientacion'] =  df_viajes.apply(lambda x: calcNSEW(x['lat_estacion_origen'], x['long_estacion_origen'], 
                                                       x['lat_estacion_destino'], x['long_estacion_destino']), axis=1)
#getting degrees
df_viajes['grados'] = df_viajes.apply(lambda x: calcGrados(x['lat_estacion_origen'], x['long_estacion_origen'], 
                                                       x['lat_estacion_destino'], x['long_estacion_destino']), axis=1)


In [16]:

#exporting csv for reporting
df_viajes.to_csv('DATA/recorridos_ecobici2019-mayo-viajes.csv')
df_estaciones.to_csv('DATA/recorridos_ecobici2019-mayo-estaciones.csv')